# Welcome to Tabular Classification/Regression

## This is the instruction for advanced user of Tabular classification/regression

In [ ]:
from wrapping import Wrapper

아래는 ALO 기본 설정 및 라이브러리 설치 코드입니다. 설치 에러가 발생하면 아래 셀을 재실행 하고, 지속적으로 문제가 있을 시 문의바랍니다

## Train workflow 

In [ ]:
# 아래는 Train 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
wrapper = Wrapper(0)

### 본 문서는 TCR train asset에 대한 활용 가이드입니다.

### input(step=0)~sampling(step=2)까지 한 번에 실행합니다.
### step을 별도 입력하지 않아도 run method 내부에서 알아서 다음 step으로 넘어갑니다.

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
## input asset
wrapper.run()

## preprocess asset
wrapper.run()

## sampling asset
wrapper.run()

### train asset args 변경하기

##### train asset의 args수정 및 확인

- 필요한경우 TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
tcr_args = wrapper.get_args(step=3)

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_list'] = ['lgb']
tcr_args

## Following A to D procedure, you can customize TCR asset's arguments option

----
### arguments explanation

|args|explanation|default|example|data type|necessary|
|---|---|---|---|---|---|
|model_type|a type of problem|`classification`|`classification, regression`|str|yes|
|data_split_method|data splitting method for train/validation dataset|`cross_validate`|`cross_validate, train_test_split`|str|no|
|evaluation_metric|evaluation metric|`accuracy`|`accuracy, f1-score, recall, precision, mse, r2, mae, rmse`|str|yes|
|model_list|HPO candidate ML model list|`[lgb, rf, cb]`|`[lgb, rf, cb, gbm, ngb]`|list|yes|
|num_hpo|number of hyperparameter setting options|`3`|`0 ~ 10`|int|no|
|param_range|range of hyperparameters for each ML model|(see below)|(see as follows)|dict|no|
|shap_ratio|sampling rate of training dataset for shapley value calculation|`1`|`0~1`|float|no|



### A. Select a type of problem: regression or classification

If you have a numeric target data, such as quantity of product, temperature, polulation etc., your problem is a regression. Otherwise, if you have a categorical target data, your problem will be a classification.

#### A-1. model_type: regression

- model_type: regression
- evaluation_meric: 과제/데이터 성격에 맞는 옵션 설정
    - mse: mean squared error
    - r2: Coefficient of determination
    - mae: mean absolute error
    - rmse: root mean squared error

- model_list: \[lgb, rf, cb, gbm, ngb\] 중 학습하고 싶은 모델 선택
    - lgb: lightGBM
    - rf: Random Forest
    - cb: CatBoost
    - gbm: Gradient Boosting
    - ngb: Natural Gradient Boosting

In [ ]:
tcr_args['model_type'] = 'regression'
tcr_args['evaluation_meric'] = 'mse'
tcr_args['model_list'] = ['lgb', 'rf', 'cb', 'gbm', 'ngb']

#### A-2. model_type: classification

- model_type: classification
- evaluation_meric: 과제/데이터 성격에 맞는 옵션 설정
    - accuracy
    - f1-score
    - recall
    - precision
- model_list: \[lgb, rf, cb, gbm(support binary only) \] 중 학습하고 싶은 모델 선택
    - lgb: lightGBM
    - rf: Random Forest
    - cb: CatBoost
    - gbm: Gradient Boosting

In [ ]:
tcr_args['model_type'] = 'classification'
tcr_args['evaluation_meric'] = 'accuracy'
tcr_args['model_list'] = ['lgb', 'rf', 'cb']

### B. Select data splitting method: train_test_split or cross_validate

Resource Usage and Performance Trade-offs for Machine Learning. If you have limited resource, then you can split just once training/validattion dataset by selecting `train_test_split` option. If you have enough resource, K-fold cross validation prevent over/under estimation of trained model so we recommend to set `cross_validate` option.

In [ ]:
# tcr_args['data_split_method'] = 'train_test_split'
tcr_args['data_split_method'] = 'cross_validate'

### C. Customize Hyperparameter Optimization

If you are professional to customize hyperparameter of each machine learning models, you may change default hyperparameter settings. Usually, higher value of n_estimators and max_depth increases model complexity and training time. If you exclude some model in `model_list`, corresponding model's setting in `param_range` will be ignored. Overall hyperparameter min-max range will split into `num_hpo` parameter options.

In [ ]:
# setting about gbm, ngb will be ignored
tcr_args['num_hpo'] = 5
tcr_args['param_range'] = {
    'rf': {'max_depth': 6, 'n_estimators': [100, 500]},
    'gbm': {'max_depth': [5, 9], 'n_estimators': [100, 500]},
    'ngb': {'col_sample': [0.4, 0.9], 'n_estimators': [100, 500]},
    'lgb': {'max_depth': [5, 9], 'n_estimators': [100, 500]},
    'cb': {'max_depth': [5, 9], 'n_estimators': [100, 500]}}

# in this case, rf's optimizing n_estimator options will be 100, 200, 300, 400, 500

### D. Set shapley_ratio

Calculating shapley value takes some time. If you don't need shapley value for training data or you only need estimated shapley value, then you can save calculating time by reducing data.

In [ ]:
# tcr_args['shap_ratio'] = 0.5
tcr_args['shap_ratio'] = 1

##### train asset 실행 

In [ ]:
wrapper.run(args=tcr_args) # 변경한 tcr_args 반영
# wrapper.data: TCR asset의 결과물입니다. 
# wrapper.config: TCR asset의 결과 config입니다. 

# tcr asset의 결과 dataframe은 wrapper.data['dataframe']으로 확인할 수 있습니다. 
wrapper.data['dataframe'].head(10) 

<br>

## Inference workflow 

In [ ]:
# 아래는 Inference 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
wrapper = Wrapper(1)

In [ ]:
# TCR inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - train(2))
## input asset
wrapper.run()

## preprocess asset
wrapper.run()

### inference asset 
##### inference asset의 args수정 및 확인
- 필요한경우 TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2))
tcr_args = wrapper.get_args(step=2)

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_type'] = 
tcr_args

### arguments explanation

- model_type: Select same option as training workflow's
- run_shapley: Calcualte shapley value for inference data or not

##### inference asset 실행 

In [ ]:
wrapper.run(args=tcr_args) # 변경한 preprocess_args 반영
# wrapper.data: TCR asset의 결과물입니다. 
# wrapper.config: TCR asset의 결과 config입니다. 

# tcr asset의 결과 dataframe은 wrapper.data['dataframe']으로 확인할 수 있습니다. 
wrapper.data['dataframe'].head(10) 